# Intro to managing assignments

All of the work managing assignments is completed in the nbgrader `<course_directory>`, while logged in as the user "instructor".  For the 2015-fall-big_data class, the `<course_directory>` is `/home/instructor/nbgrader/courses/2015-fall-big_data`.  This IPython notebook, `manage_assignments.ipynb`, should be run from the `<course_directory>`.  Do not try to run it in any other directory.

This notebook contains:

- instructions on initializing the course's gradebook database.
- an overview of the process of making, releasing, collecting, and grading assignments.
- tips for troubleshooting problems with assignments.

## Table of Contents

- [Initialize gradebook database](#Initialize-gradebook-database)

    - [Add students to the gradebook](#Add-students-to-the-gradebook)
    - [Add an assignment to the gradebook](#Add-an-assignment-to-the-gradebook)
    
- [Assignment workflow after adding to gradebook](#Assignment-workflow-after-adding-to-gradebook)

    - [Implementing assignments](#Implementing-assignments)
    - [1. render student version - `nbgrader assign "<assignment>"`](#1.-render-student-version---nbgrader-assign-"<assignment>")
    - [2. release student version - `nbgrader release "<assignment>"`](#2.-release-student-version---nbgrader-release-"<assignment>")
    - [3. STUDENT - Find assignments - `nbgrader list`](#3.-STUDENT---Find-assignments---nbgrader-list)
    - [4. STUDENT - Fetch assignment - `nbgrader fetch "<assignment>"`](#4.-STUDENT---Fetch-assignment---nbgrader-fetch-"<assignment>")
    - [5. STUDENT - Submit assignment - `nbgrader submit "<assignment>"`](#5.-STUDENT---Submit-assignment---nbgrader-submit-"<assignment>")
    - [6a. STUDENT - List submitted assignments - `nbgrader list --cached`](#6a.-STUDENT---List-submitted-assignments---nbgrader-list---cached)
    - [6b. List assignments with submissions - `nbgrader list --inbound`](#6b.-List-assignments-with-submissions---nbgrader-list---inbound)
    - [7. Collect assignments - `nbgrader collect "<assignment>"`](#7.-Collect-assignments---nbgrader-collect-"<assignment>")
    - [8. Auto-grade assignments - `nbgrader autograde "<assignment>"`](#8.-Auto-grade-assignments---nbgrader-autograde-"<assignment>")
    - [9. Grade using the Formgrade application - `nbgrader formgrade`](#9.-Grade-using-the-Formgrade-Application---nbgrader-formgrade)
    
        - [Setting up, starting, and stopping the Formgrade application](#Setting-up,-starting,-and-stopping-the-Formgrade-application)
    
    - [10. Generating feedback for students - `nbgrader feedback "<assignment>"`](#10.-Generating-feedback-for-students---nbgrader-feedback-"<assignment>")
    - [11. Getting feedback back to student](#11.-Getting-feedback-back-to-student)
    
- [Troubleshooting](#Troubleshooting)

    - [`nbgrader formgrade` issues](#nbgrader-formgrade-issues)
    
        - [CONFIGPROXY_AUTH_TOKEN mismatch between jupyterhub and formgrade](#CONFIGPROXY_AUTH_TOKEN-mismatch-between-jupyterhub-and-formgrade)
        - ["attempt to write a readonly database"](#"attempt-to-write-a-readonly-database")
        - ["unable to open database file" api_tokens](#"unable-to-open-database-file"-api_tokens)
        - [URLs on formgrade pages refer to proxy IP address, not public IP or domain name](#URLs-on-formgrade-pages-refer-to-proxy-IP-address,-not-public-IP-or-domain-name)
        - [Notebook file not found when you try to grade a given submission](#Notebook-file-not-found-when-you-try-to-grade-a-given-submission)
        
    - [notebook issues](#notebook-issues)
    
        - [nbgrader solution can not be in a normal cell](#nbgrader-solution-can-not-be-in-a-normal-cell)
        - [grade cell point value must be greater than 0](#grade-cell-point-value-must-be-greater-than-0)
        - [make sure you run code cells that just contain a function definition before calling the function](#make-sure-you-run-code-cells-that-just-contain-a-function-definition-before-calling-the-function)
        
    - [gradebook database issues](#gradebook-database-issues)
    
        - [corruption of tables that hold auto-grading](#corruption-of-tables-that-hold-auto-grading)

# Initialize gradebook database

- Back to the [Table of Contents](#Table-of-Contents)

A given course's students, assignments, and grades are stored in that course's gradebook database.  This database file is stored at `<course_directory>/gradebook.db`.  Before you do any work with assignments, you need to initialize the gradebook database by adding students and assignments to it.

## Add students to the gradebook

- Back to the [Table of Contents](#Table-of-Contents)

Before you do anything else, you'll need to add all your students to the gradebook.

Assumptions:

- as long as the "source" directory is in the same directory as this page, everything should be fine.
- you'll need to make one line per student.  If you forget a student and then try to grade, there will be errors, so enter all your students right from the start.

In [ ]:
# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook

# connect to the database.
gb = Gradebook("sqlite:///gradebook.db")

# add some students to the database
# template:
# gb.add_student( "<unix_username>", first_name = "<first_name>", last_name = "<last_name>", email = "<email>" )

## Add an assignment to the gradebook

- Back to the [Table of Contents](#Table-of-Contents)

Use the code below to add an assignment to the gradebook.  You should just need to set the assignment_name and assignment_due_date fields.  More notes on other options TK.

Assumptions:

- as long as the "source" directory is in the same directory as this page, everything should be fine.
- should not need to list out the ipython notebooks in the assignment - you are just telling it the name of the assignment (which is also the name of the folder in "source" in which the assignment's notebooks live).

In [ ]:
import os

# remove an existing database
#if os.path.exists("gradebook.db"):
#    os.remove("gradebook.db")

# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook
gb = Gradebook("sqlite:///gradebook.db")

#==================================
# set up assignment call variables
#==================================

# assignment_name - the name of the folder inside the "source" folder that contains
#    the ipython notebooks for a given assignment.
# example: assignment_name = "08. Machine Learning"

# assignment_due_date - the date the assignment is due, in format "YYYY-MM-DD HH:MM:SS:MMMMMM TZ"
# example: assignment_due_date = "2015-02-01 17:00:00.000000 PST"

# add the assignment to the database and specify a due date
gb.add_assignment( assignment_name, duedate = assignment_due_date )

<hr />

# Assignment workflow after adding to gradebook

- Back to the [Table of Contents](#Table-of-Contents)

Once you've run the code above to add an assignment to the gradebook and put your assignments in the `source` folder, you'll use the `nbgrader` command to:

- make a student version of the assignment
- release it to the students
- check for submissions
- gather submissions for grading
- autograde all assignments (even those with no auto-grade questions, just so you can use the form grade app).
- start the formgrade app to grade assignments that can not be auto-graded.

Details on each step of this process follow, using an example assignment named either "06. Networks" or "02. Database Basics".

## Implementing assignments

- Back to the [Table of Contents](#Table-of-Contents)

Before starting the formal process of assigning and grading an assignment, you'll need to get the assignment code into the `source` directory inside the course directory.  For our class, the source folder is a github repository ( [https://github.com/CSSIP-AIR/Big-Data-Workbooks](https://github.com/CSSIP-AIR/Big-Data-Workbooks) ), cloned into the course folder with its root directory renamed to `source`.

To initially clone this repository, you'd run the following command in the course folder:

    git clone https://github.com/CSSIP-AIR/Big-Data-Workbooks.git source
    
As assignments are developed, updates should be committed to the github repository.  Once they are ready to be processed by nbgrader, go into the `source` folder in a command shell and run:

    git pull
    
This will pull down the latest code from the repository.

The source folder is structured so each _assignment_ is in a folder inside `source`, and then each _exercise_ for that _assignment_ is a Jupyter notebook within the _assignment_ folder.  You can include other files and folders in the _assignment_ folder, and they will be distributed to students as part of the assignment.

To develop an assignment, you'll at the least need to download the github repository and then install nbgrader and the nbgrader toolbar in your development computer's jupyter setup.  First get the notebook as you want it, then turn on the nbgrader "Create assignment" toolbar and go through and set up the question and answer cells as needed.  Once you are done:

- Clear out all cell output --> In the "Cell" menu, choose "All Output", then "Clear".
- In "File" menu, "Save and Checkpoint" your assignment.
- In "File" meny, "Close and Halt" the notebook.
- in command shell, cd into the "`source`" directory.
- use git to add and commit changes, then pull, then push your changes to github, so they can be pulled down by the instructor.

More information on nbgrader-izing an assignment:

- [http://nbgrader.readthedocs.org/en/stable/user_guide/02_developing_assignments.html](http://nbgrader.readthedocs.org/en/stable/user_guide/02_developing_assignments.html)

## 1. render student version - `nbgrader assign "<assignment>"`

- Back to the [Table of Contents](#Table-of-Contents)

Use the `nbgrader assign "<assignment>"` command to render the student version of the notebook:

    nbgrader assign "06. Networks"
    
- Example output:
    
        $ nbgrader assign "06. Networks"
        Networks" --IncludeHeaderFooter.header=source/Style\ Guide.ipynb
        [AssignApp | INFO] Converting notebook source/./06. Networks/networks_exercise.ipynb to notebook
        [AssignApp | INFO] Writing 11791 bytes to release/./06. Networks/networks_exercise.ipynb
        [AssignApp | INFO] Setting destination file permissions to 644  
        
After the first time you do this, to re-render, then overwrite the existing student version of the assignment:
    
    nbgrader assign "06. Networks" --force
        
- Example output:
    
        $ nbgrader assign "06. Networks" --force
        [AssignApp | WARNING] Removing existing assignment: release/06. Networks
        [AssignApp | INFO] Converting notebook source/./06. Networks/networks_exercise.ipynb to notebook
        [AssignApp | INFO] Writing 11791 bytes to release/./06. Networks/networks_exercise.ipynb
        [AssignApp | INFO] Setting destination file permissions to 644  
        
- Example output if you forget the `--force`:

        $ nbgrader assign "06. Networks"
        [AssignApp | INFO] Skipping existing assignment: release/06. Networks

If you have a header (we don't at the moment):
    
    nbgrader assign "06. Networks" --IncludeHeaderFooter.header=source/Style\ Guide.ipynb
    
Running `nbgrader assign "<assignment>"` on an assignment renders student versions of the assignment's notebooks, then  places the student versions in a directory named the same as the assignment in the `<course_folder>/release` folder.

## 2. release student version - `nbgrader release "<assignment>"`

- Back to the [Table of Contents](#Table-of-Contents)

Use the `nbgrader release "<assignment>"` command to place the assignment in the `exchange` folder:

    nbgrader release "08. Machine Learning"
    
- Example output:

        $ nbgrader release "06. Networks"
        [ReleaseApp | INFO] Source: /home/instructor/nbgrader/courses/2015-fall-big_data/release/06. Networks
        [ReleaseApp | INFO] Destination: /srv/nbgrader/exchange/2015-fall-big_data/outbound/06. Networks
        [ReleaseApp | INFO] Released as: 2015-fall-big_data 06. Networks
        
After the first time you do this, to overwrite the existing copy of the assignment in the exchange:
    
    nbgrader release "06. Networks" --force
    
- Example output:

        $ nbgrader release "06. Networks" --force
        [ReleaseApp | INFO] Overwriting files: 2015-fall-big_data 06. Networks
        [ReleaseApp | INFO] Source: /home/instructor/nbgrader/courses/2015-fall-big_data/release/06. Networks
        [ReleaseApp | INFO] Destination: /srv/nbgrader/exchange/2015-fall-big_data/outbound/06. Networks
        [ReleaseApp | INFO] Released as: 2015-fall-big_data 06. Networks

- Example output if you forget the `--force`:

        $ nbgrader release "06. Networks"
        [ReleaseApp | ERROR] Destination already exists, add --force to overwrite: 2015-fall-big_data 06. Networks
        
Releasing an assignment places all of that assignment's notebooks in `<exchange_folder>/<class>/outbound/<assignment>/`.  So, for the default exchange directory path ("/srv/nbgrader/exchange"), class "2015-fall-big_data" and assignment "06. Networks", the assignment's notebooks would be placed in:

    /srv/nbgrader/exchange/2015-fall-big_data/outbound/06. Networks/

## 3. STUDENT - Find assignments - `nbgrader list`

- Back to the [Table of Contents](#Table-of-Contents)

Once the assignment has been released, the student can see an assignment has been released by either:

- looking in the "Released assignments" section of the "Assignments" tab.
    
- OR checking the output of the nbgrader `nbgrader list` command:

        nbgrader list
        
    - Example output:
    
            $ nbgrader list
            [ListApp | INFO] Released assignments:
            [ListApp | INFO] 2015-fall-big_data 06. Networks

## 4. STUDENT - Fetch assignment - `nbgrader fetch "<assignment>"`

- Back to the [Table of Contents](#Table-of-Contents)

Students can download a released assignment to work on it by either:

- clicking on the "Fetch" button next to the asignment in the **"Released assignments"** section of the **"Assignments"** tab.

- OR running the `nbgrader fetch "<assignment>"` command:
    
        nbgrader fetch "08. Machine Learning"
     
    - Example output:
    
            $ nbgrader fetch "06. Networks"
            [FetchApp | INFO] Source: /srv/nbgrader/exchange/2015-fall-big_data/outbound/06. Networks
            [FetchApp | INFO] Destination: /home/jmorgan/Downloads/06. Networks
            [FetchApp | INFO] Fetched as: 2015-fall-big_data 06. Networks
            
If you use the `nbgrader fetch` command, your assignments will be downloaded into the folder in which you run the command.  The "Fetch" button will always download your assignments into your home folder.  If you don't want the assignments downloaded directly into your home folder, you must use the command line command rather than the Assignments tab, as the Assignments tab doesn't know to look anywhere other than your home folder for assignments.

## 5. STUDENT - Submit assignment - `nbgrader submit "<assignment>"`

- Back to the [Table of Contents](#Table-of-Contents)

Once the student has completed the notebook(s) that make up the assignment, they can turn the assignment in by either:

- clicking on the "Submit" button next to the assignment in the "Downloaded assignments" section of the "Assignments" tab.
- OR running the `nbgrader submit "<assignment>"` command:
    
        nbgrader submit "06. Networks"
            
If you used the `nbgrader fetch "<assignment>"` command to fetch an assignment to a folder other than your home folder, you will have to use the `nbgrader submit "<assignment>"` command in the folder where you downloaded the assignment to submit it, again because the Assignments tab doesn't know to look anywhere other than your home folder for assignments.

After a student submits their assignment, the updated notebook(s) are stored in `<exchange_folder>/inbound`, in a folder named `<student_user>+<assignment>+<timestamp>` (example: `jmorgan+06. Networks+2015-08-29 23:33:29 UTC`).  Inside, each notebook submitted by the student for the assignment is stored, as well as a file named `timestamp.txt` that contains the same time stamp as is appended to the folder name.

## 6a. STUDENT - List submitted assignments - `nbgrader list --cached`

- Back to the [Table of Contents](#Table-of-Contents)

A student can see which assignments they have submitted using the `nbgrader list` command:

    nbgrader list --cached
        
- Example output (course, then username, then assignment, then submission date):

        $ nbgrader list --cached
        [ListApp | INFO] Submitted assignments:
        [ListApp | INFO] 2015-fall-big_data jmorgan 06. Networks 2015-08-29 23:33:29 UTC

## 6b. List assignments with submissions - `nbgrader list --inbound`

- Back to the [Table of Contents](#Table-of-Contents)

To see which assignments have submissions which can be graded, in the courses directory, the instructor can use the `nbgrader list` command:

    nbgrader list --inbound
        
- Example output (course, then username, then assignment, then submission date):

        $ nbgrader list --inbound
        [ListApp | INFO] Submitted assignments:
        [ListApp | INFO] 2015-fall-big_data jmorgan 06. Networks 2015-08-29 23:33:29 UTC


## 7. Collect assignments - `nbgrader collect "<assignment>"`

- Back to the [Table of Contents](#Table-of-Contents)

To collect submitted assignments for grading, after you see submissions in `nbgrader list --inbound`, use the `nbgrader collect "<assignment>"` command to collect assignments back into the instructor's grading area from the exchange folder:

     nbgrader collect "06. Networks"
        
- Example output:

        $ nbgrader collect "06. Networks"
        [CollectApp | INFO] Collecting submission: jmorgan 06. Networks
        
After running `nbgrader collect "<assignment>"`, the collected assignments are stored in the `<course_directory>/submitted` folder.  Inside, each student has a directory that matches their unix username.

Each student's directory contains a folder for each assignment.  Inside each assignment folder are the notebooks that were collected and a timestamp file that holds the date and time the assignment was submitted.

After the first time you do this, if students have resubmitted, the output will look like this:

    $ nbgrader collect "02. Database Basics"
    [CollectApp | INFO] Submission already exists, use --update to update: kwilken 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: wchang 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: acaporaso 02. Database Basics
    [CollectApp | INFO] Collecting submission: adelano 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: smcdonald 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: eross 02. Database Basics
    [CollectApp | INFO] Collecting submission: pmiller 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: jmorgan 02. Database Basics
    [CollectApp | INFO] Collecting submission: ggraham 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: thill 02. Database Basics
    [CollectApp | INFO] Collecting submission: dglennon 02. Database Basics
    [CollectApp | INFO] Collecting submission: coslund 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: wkingkade 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: nczaplicki 02. Database Basics
    [CollectApp | INFO] Collecting submission: azotti 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: jmenza 02. Database Basics
    [CollectApp | INFO] Collecting submission: dbryant 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: mbenetsky 02. Database Basics
    [CollectApp | INFO] Submission already exists, use --update to update: ebyerly 02. Database Basics
    [CollectApp | INFO] Collecting submission: cwurster 02. Database Basics


To overwrite the existing copy of the assignment with the latest:

    nbgrader collect "06. Networks" --update

- Example output:

        $ nbgrader collect "02. Database Basics" --update
        [CollectApp | INFO] No newer submission to collect: thill 02. Database Basics
        [CollectApp | INFO] Updating submission: eross 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: dbryant 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: cwurster 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: azotti 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: wkingkade 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: pmiller 02. Database Basics
        [CollectApp | INFO] Updating submission: smcdonald 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: jmorgan 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: ebyerly 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: dglennon 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: acaporaso 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: jmenza 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: coslund 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: wchang 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: kwilken 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: mbenetsky 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: adelano 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: ggraham 02. Database Basics
        [CollectApp | INFO] No newer submission to collect: nczaplicki 02. Database Basics
        
As you can see, just because a student has an existing submission doesn't mean that they have submitted a new version.  Update is smart enough to only update if there is a submission newer than the one already stored for the student.

## 8. Auto-grade assignments - `nbgrader autograde "<assignment>"`

- Back to the [Table of Contents](#Table-of-Contents)

After you have collected some assignments, you will autograde them using the `nbgrader autograde "<assignment>"` command:

    nbgrader autograde "06. Networks"
    
- Example output:

        [AutogradeApp | INFO] Copying submitted/jmorgan/06. Networks/timestamp.txt -> autograded/jmorgan/06. Networks/timestamp.txt
        [AutogradeApp | INFO] SubmittedAssignment<06. Networks for jmorgan> submitted at 2015-08-29 23:33:29
        [AutogradeApp | INFO] Overwriting files with master versions from the source directory
        [AutogradeApp | INFO] Sanitizing submitted/jmorgan/06. Networks/networks_exercise.ipynb
        [AutogradeApp | INFO] Converting notebook submitted/jmorgan/06. Networks/networks_exercise.ipynb to notebook
        [AutogradeApp | INFO] Writing 8008 bytes to autograded/jmorgan/06. Networks/networks_exercise.ipynb
        [AutogradeApp | INFO] Autograding autograded/jmorgan/06. Networks/networks_exercise.ipynb
        [AutogradeApp | INFO] Converting notebook autograded/jmorgan/06. Networks/networks_exercise.ipynb to notebook
        [AutogradeApp | INFO] Executing notebook with kernel: python2
        [AutogradeApp | INFO] Writing 10097 bytes to autograded/jmorgan/06. Networks/networks_exercise.ipynb
        [AutogradeApp | INFO] Setting destination file permissions to 444

You must auto-grade all assignments, even those without auto-graded questions, since the form grade app also depends on the output of this command.

Once assignments are auto-graded, they are stored in the `<course_directory>/autograded` folder.  Inside, each student has a directory that matches their unix username.

Each student's directory contains a folder for each assignment.  Inside each assignment folder are the notebooks that were collected and a timestamp file that holds the date and time the assignment was auto-graded.

**_ NOTE: For classes with 10 or more students, auto-grading can take a few minutes.  If you are SSHed in to the server on which you are grading, it is probably a good idea to run the autograde process in a GNU screen session, so it will continue if you are disconnected._**

After the first time you do this, to re-autograde (overwrite the existing auto-graded version of the assignment):
    
    nbgrader autograde "07. mini Machine Learning" --force
        
- Example output:
    
        instructor@ip-172-31-36-239:~/nbgrader/courses/2015-fall-big_data$ nbgrader autograde "07. mini Machine Learning" --force
        [AutogradeApp | WARNING] Removing existing assignment: autograded/jmorgan/07. mini Machine Learning
        [AutogradeApp | INFO] Copying submitted/jmorgan/07. mini Machine Learning/timestamp.txt -> autograded/jmorgan/07. mini Machine Learning/timestamp.txt
        [AutogradeApp | INFO] Copying submitted/jmorgan/07. mini Machine Learning/asgt7_cleaned.msg -> autograded/jmorgan/07. mini Machine Learning/asgt7_cleaned.msg
        [AutogradeApp | INFO] SubmittedAssignment<07. mini Machine Learning for jmorgan> submitted at 2015-11-09 17:40:07
        [AutogradeApp | INFO] Overwriting files with master versions from the source directory
        [AutogradeApp | INFO] Sanitizing submitted/jmorgan/07. mini Machine Learning/Machine Learning.ipynb
        [AutogradeApp | INFO] Converting notebook submitted/jmorgan/07. mini Machine Learning/Machine Learning.ipynb to notebook
        [AutogradeApp | INFO] Writing 71417 bytes to autograded/jmorgan/07. mini Machine Learning/Machine Learning.ipynb
        [AutogradeApp | INFO] Autograding autograded/jmorgan/07. mini Machine Learning/Machine Learning.ipynb
        [AutogradeApp | INFO] Converting notebook autograded/jmorgan/07. mini Machine Learning/Machine Learning.ipynb to notebook
        [AutogradeApp | INFO] Executing notebook with kernel: python2
        [AutogradeApp | INFO] Writing 100207 bytes to autograded/jmorgan/07. mini Machine Learning/Machine Learning.ipynb
        [AutogradeApp | INFO] Setting destination file permissions to 444
        
- Example output if you forget the `--force`:

        instructor@ip-172-31-36-239:~/nbgrader/courses/2015-fall-big_data$ nbgrader autograde "07. mini Machine Learning"
        [AutogradeApp | INFO] Skipping existing assignment: autograded/jmorgan/07. mini Machine Learning

### Troubleshooting

If your students create backup copies of notebooks in their assignment folders, `nbgrader autograde ...` will crash when it encounters an ipython notebook in the directory that it isn't expecting.  It might also crash for other reasons...  To troubleshoot:

#### `--notebook`

One troubleshooting strategy is to explicitly limit auto-grading to specific notebooks.  For example, for an assignment "02. Database Basics", which in its initial form had one nbgrader notebook (`Data_and_database.ipynb`) and lots of other files and folders, and for which some students who struggled create multiple backup copies of this IPython Notebook with different names, all that is fine as long as you use the `--notebook` parameter to tell nbgrader to just grade `Data_and_databases.ipynb`.  

Syntax:

    nbgrader autograde "<assignment_name>" --notebook "<notebook_name_pattern>"
    
WHERE:

- `<assignment_name>` is the assignment you are grading.
- `<notebook_name_pattern>` is a file system name match pattern like those used in unix where an asterisk ( "`*`" ) matches zero to many characters anywhere you place it in a pattern string.  This will be compared to the complete name of each file in the assignment directory (so must account for the extension as well - like `*.ipynb`).  Example: only files that start with "Data" and end with ".ipynb" - "`Data*.ipynb`".
    
Example command for scenario above (assignment "02. Database Basics", limit to notebook "Data_and_databases.ipynb"):

    nbgrader autograde "02. Database Basics" --notebook "Data_and_databases.ipynb"
    
#### `--student`

Another troubleshooting strategy is to just autograde a given student, using the `--student` parameter.

Syntax:

    nbgrader autograde "<assignment_name>" --student "<student_username>"
    
WHERE:

- `<student_username>` is the username of the student, from when they were added to the gradebook.

Example:

    nbgrader autograde "02. Database Basics" --student "dglennon"

## 9. Grade using the Formgrade Application - `nbgrader formgrade`

- Back to the [Table of Contents](#Table-of-Contents)

To access the formgrade application, you will combine the URL you use to access the base jupyterhub and combine it with the path from the "Proxying" line above.  For example, for jupyterhub configured to be non-ssl, with domain of "bigdataforsocialscience.com" and hub listening on port 8000:

- [http://bigdataforsocialscience.com:8000/hub/nbgrader/2015-fall-big_data](http://bigdataforsocialscience.com:8000/hub/nbgrader/2015-fall-big_data)

For the same, but with SSL certificates enabled:

- [https://bigdataforsocialscience.com:8000/hub/nbgrader/2015-fall-big_data](https://bigdataforsocialscience.com:8000/hub/nbgrader/2015-fall-big_data)

The formgrade application must be started and run by the instructor user.  Once it is up and running, though, any user whose username is in the list of `c.HubAuth.graders` in `<course_folder>/nbgrader_config.py` can use the URL to connect and grade.

Configuring the formgrade application can be tricky.  For more information on troubleshooting, see "`nbgrader formgrade` issues" in the troubleshooting section below.

For more information on using the formgrade application, see the help located within the formgrade application itself.

### Setting up, starting, and stopping the Formgrade application

- Back to the [Table of Contents](#Table-of-Contents)

In general, you'll leave the Formgrade application running on your server.  Initially, though, you'll need to configure it and set it up.  Here's how to do that.

Before you start the formgrade app, you need to have its properties configured in the nbgrader_config.py file that is in the same folder as this file.  For more details, see [http://nbgrader.readthedocs.org/en/stable/user_guide/11_jupyterhub_config.html#configuring-nbgrader-formgrade](http://nbgrader.readthedocs.org/en/stable/user_guide/11_jupyterhub_config.html#configuring-nbgrader-formgrade).

To start the formgrade app while logged in as the instructor user:

- open a screen session in which you'll let the app run:

        screen
        
- make sure you are in the `<course_directory>`.

        pwd

- export the same `CONFIGPROXY_AUTH_TOKEN` environment variable value as is used to start the jupyterhub itself:

        export CONFIGPROXY_AUTH_TOKEN='<CONFIGPROXY_AUTH_TOKEN>'
    
- run `nbgrader formgrade`:

        nbgrader formgrade
        
    - example output:
    
            $ nbgrader formgrade
            [FormgradeApp | INFO] Proxying /hub/nbgrader/2015-fall-big_data --> http://127.0.0.1:9000
            [FormgradeApp | INFO] Serving MathJax from /usr/local/lib/python3.4/dist-packages/notebook/static/components/MathJax/
            [FormgradeApp | INFO] Form grader running at http://127.0.0.1:9000/
            [FormgradeApp | INFO] Use Control-C to stop this server
            * Running on http://127.0.0.1:9000/ (Press CTRL+C to quit)

- detach from the screen session by pressing "Control" and "A" together, then pressing "D" once you release Control and A.

At this point, the FormGrade application is running in the background in this screen session.

To stop it:

- log into the server as the instructor user.
- reconnect to the screen session:

        screen -r
        
- Press "Control" and "C" together to send the kill signal to the application.

To start it again:

- run `nbgrader formgrade`:

        nbgrader formgrade

If your server is restarted, you'll need to go through all of the startup steps again to start it up.

## 10. Generating feedback for students - `nbgrader feedback "<assignment>"`

- Back to the [Table of Contents](#Table-of-Contents)

Once you have graded and provided feedback on assignments with the formgrade application, you can then generate HTML files that contain your feedback and return them to the students using the `nbgrader feedback "<assignment>"` command:

    nbgrader feedback "06. Networks"

- Example output:

        $ nbgrader feedback "06. Networks"
        [FeedbackApp | INFO] Copying autograded/jmorgan/06. Networks/timestamp.txt -> feedback/jmorgan/06. Networks/timestamp.txt
        [FeedbackApp | INFO] Converting notebook autograded/jmorgan/06. Networks/networks_exercise.ipynb to html
        [FeedbackApp | INFO] Writing 210591 bytes to feedback/jmorgan/06. Networks/networks_exercise.html
        [FeedbackApp | INFO] Setting destination file permissions to 444

After the first time you do this, to overwrite the existing copy of the assignment's feedback in both the course folder and in the exchange:
    
    nbgrader feedback "06. Networks" --force
    
- Example output:

        $ nbgrader feedback "06. Networks" --force
        [FeedbackApp | WARNING] Removing existing assignment: feedback/jmorgan/06. Networks
        [FeedbackApp | INFO] Copying autograded/jmorgan/06. Networks/timestamp.txt -> feedback/jmorgan/06. Networks/timestamp.txt
        [FeedbackApp | INFO] Converting notebook autograded/jmorgan/06. Networks/networks_exercise.ipynb to html
        [FeedbackApp | INFO] Writing 210591 bytes to feedback/jmorgan/06. Networks/networks_exercise.html
        [FeedbackApp | INFO] Setting destination file permissions to 444

- Example output if you forget the `--force`:

        $ nbgrader feedback "06. Networks"
        [FeedbackApp | INFO] Skipping existing assignment: feedback/jmorgan/06. Networks
i
        
After running `nbgrader feedback "<assignment>"`, all autograded notebooks for the assignment are converted to HTML, including feedback, and the HTML is moved to the `<course_directory>/feedback` folder, which is structured just like the autograded folder, except the assignments have HTML feedback rather than notebooks.

Inside, each student has a directory that matches their unix username.  Each student's directory contains a folder for each assignment.  Inside each assignment folder are the HTML rendering of the assignment's notebooks that include grading and feedback, and a timestamp file that holds the date and time the assignment was graded.

## 11. Getting feedback back to student

- Back to the [Table of Contents](#Table-of-Contents)

Run the following code to turn HTML feedback into PDFs and send PDF feedback to each user.

Before you run this:

- Make sure that the packages `texlive-full` and `pandoc` are installed on your system:

         (sudo) apt-get install texlive-full
         (sudo) apt-get install pandoc

- Make sure to set the following variables in the code below:

### nbgrader settings

- **`nbgrader_home`** = to the full path to the nbgrader home.
- **`assignment_name`** = to the name of the assignment whose feedback you want to generate and email.

### SMTP server settings

- **`smtp_host`** = string hostname of SMTP server we'll use to send feedback.
- **`smtp_port`** = port number of SMTP server (587 is default for TLS).
- **`smtp_host_username`** = string username for SMTP server authentication.
- **`smtp_host_password`** = string password for SMTP server authentication.

### email message settings (optional)

- **`message_from`** = email address from which the message is being sent.
- **`message_subject`** = String subject of the message - by default, is "SURV 699Y Fall 2015 Course - Feedback for " + assignment_name
- **`message_body`** = contents of the message you want to accompany the feedback PDFs.
- **`custom_message_dict[ '<username>' ]`** = a custom message you want for a given student.
- **`message_signature`** = signature string, which will follow message and optional custom message.
- **`message_bcc`** = email addresses you want blind-copied on each message.

In [ ]:
#==============================================================================#
# imports
#==============================================================================#

import mimetypes
import os
import sqlite3
import subprocess
import sys
import traceback

# imports - sending email
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

#==============================================================================#
# declare variables
#==============================================================================#

nbgrader_home = ""
assignment_name = ""
current_directory_path = ""
feedback_path = ""
my_connection = None
my_cursor = None
sql_string = ""
result_set = None
student_counter = -1
current_row = None

# declare variables - SMTP server
smtp_host = ''
smtp_host_username = ''
smtp_host_password = ''
smtp_port = 587
smtp_server = None
smtp_send_result = None

# declare variables - email message
multipart_message = ""
message_from = ""
message_subject = ""
message_body = ""
custom_message_dict = {}
message_signature = ""
complete_message_body = ""
message_bcc = ""

# declare variables - process assignment
assignment_result_set = None
assignment_count = -1
assignment_row = None
assignment_id = ""
notebook_result_set = None
notebook_row = None
notebook_name = ""
assignment_notebook_list = []
is_assignment_ok = True

# declare variables - process student
student_username = ""
student_email = ""
student_first_name = ""
student_last_name = ""
student_feedback_path = ""
pandoc_command_list = []
pandoc_command_result = None
#file_list = []
file_name = ""
#file_name_lower = ""
pdf_file_path = ""
pdf_file_name_list = []
pdf_file_count = -1
content_type = ""
main_type = ""
sub_type = ""
encoding = ""
text_message = None
pdf_message = None

#==============================================================================#
# initialize variables
#==============================================================================#

#------------------------------------------------------------------------------#
# == initialize variables - nbgrader
#------------------------------------------------------------------------------#

# example: nbgrader_home = "/home/instructor/nbgrader/courses/2015-fall-big_data"
nbgrader_home = ""
# example: assignment_name = "02. Database Basics"
assignment_name = ""

#------------------------------------------------------------------------------#
# == initialize variables - email message
#------------------------------------------------------------------------------#

message_from = ""
message_subject = "Feedback for " + assignment_name
message_body = """Hello,

Attached is a PDF with feedback on assignment """
message_body += assignment_name
message_body += """ (it is a copy of your assignment, with “Comments” after each answer).  Please let me know if you have any questions.
"""
#custom_message_dict[ "jmorgan" ] = "test custom message!"
message_signature = """
Thanks!

Jon
"""
message_bcc = ""

#------------------------------------------------------------------------------#
# == initialize variables - SMTP server
#------------------------------------------------------------------------------#

smtp_host = ''
smtp_host_username = ''
smtp_host_password = ''
smtp_port = 587
# smtp_use_TLS = True - always use TLS.

#==============================================================================#
# Process feedback
#==============================================================================#

# get current working directory, use that to make path to feedback directory.
current_directory_path = os.getcwd()
feedback_path = current_directory_path + "/feedback"

print( "Feedback path = " + feedback_path )

# must run this from nbgrader_home.
if( nbgrader_home == current_directory_path ):

    # must have an assignment.
    if ( ( assignment_name is not None ) and ( assignment_name != "" ) ):

        # use try-->except-->finally to make sure you always close your database
        #    connections.
        try:

            # make a connection - pass the name of the database file.
            my_connection = sqlite3.connect( 'gradebook.db' )

            # set row_factory that returns values mapped to column names
            #   as well as in an ordered list
            my_connection.row_factory = sqlite3.Row

            # then, make a cursor.
            my_cursor = my_connection.cursor()

            #------------------------------------------------------------------#
            # Assignment processing.
            #------------------------------------------------------------------#
            
            # set assignment OK flag to True to start.
            is_assignment_ok = True
            
            # retrieve assignment record for assignment name.
            sql_string = "SELECT * FROM assignment WHERE name = ?;"
            assignment_result_set = my_cursor.execute( sql_string, ( assignment_name, ) )
            
            # get assignment ID.
            assignment_count = 0
            for assignment_row in assignment_result_set:
                
                # increment count
                assignment_count += 1
                assignment_id = assignment_row[ "id" ]
                
                print( "--> Found assignment ID \"" + assignment_id + "\"" )
                
            #-- END loop over records to get ID. --#
            
            # how many?
            if ( assignment_count == 1 ):
            
                # 1 - right answer.
                
                # Retrieve names of the notebooks that are formally part of the assignment.
                sql_string = "SELECT * FROM notebook WHERE assignment_id = ?;"
                notebook_result_set = my_cursor.execute( sql_string, ( assignment_id, ) )
                
                # loop over results, storing each name in assignment_notebook_list.
                for notebook_row in notebook_result_set:
                    
                    # retrieve name.
                    notebook_name = notebook_row[ "name" ]
                    
                    print( "----> Found notebook \"" + notebook_name + "\"" )
                    
                    # add to list.
                    assignment_notebook_list.append( notebook_name )
                    
                #-- END loop over notebooks. --#
                
                # got at least 1 notebook name?
                if ( len( assignment_notebook_list ) <= 0 ):
                    
                    # no notebooks.  Nothing to do here.
                    print( "No Notebooks for assignment " + assignment_name + " - can not proceed." )
                    is_assignment_ok = False
                    
                #-- END check to see if notebooks found. --#

            else:
                
                # not 1.  0?
                if ( assignment_count == 0 ):
                    
                    # 0 - unknown assignment.  Error.
                    print( "Unknown assignemnt: " + assignment_name + " - can not proceed." )
                    is_assignment_ok = False
                    
                elif ( assignment_count > 1 ):
                    
                    # more than 1.  Error.
                    print( "Multiple matches for assignment: " + assignment_name + " - can not proceed." )
                    is_assignment_ok = False
                
                else:
                    
                    # -1?  Error.
                    print( "Unknown error for assignment: " + assignment_name + " - count less than 0.  Can not proceed." )
                    is_assignment_ok = False
                    
                #-- END check to see what is going on with result not equal to 1. --#
                
            #-- END check to see if 1 assignment returned. --#
                    
            # is assignment OK?
            if ( is_assignment_ok == True ):
            
                # initialize query to count loop over students.
                sql_string = "SELECT * FROM student ORDER BY last_name ASC, first_name ASC;"
                #sql_string = "SELECT * FROM student WHERE id IN ( 'jmorgan' ) ORDER BY last_name ASC, first_name ASC;"
                #sql_string = "SELECT * FROM student WHERE id NOT IN ( 'cjones', 'aemad' ) ORDER BY last_name ASC, first_name ASC;"

                # Actually use cursor here...
                result_set = my_cursor.execute( sql_string )

                # output results
                print( "Student list:" )

                # open SMPT server for sending email.
                with smtplib.SMTP( host = smtp_host, port = smtp_port ) as smtp_server:

                    # authenticate and start TLS
                    smtp_server.starttls()
                    smtp_server.login( smtp_host_username, smtp_host_password )

                    # loop over students in result set.
                    student_counter = 0
                    for current_row in result_set:

                        # increment student counter
                        student_counter += 1

                        # initialize variables
                        student_username = ""
                        student_email = ""
                        student_first_name = ""
                        student_last_name = ""
                        pandoc_command_list = []
                        pdf_file_name_list = []
                        pdf_file_count = -1

                        print( "\n\n==> Student " + str( student_counter ) + " = " + current_row[ "first_name" ] + " " + current_row[ "last_name" ] + " ( " + current_row[ "id" ] + " ) - " + current_row[ "email" ] )

                        # get student information
                        student_username = current_row[ "id" ]
                        student_email = current_row[ "email" ]
                        student_first_name = current_row[ "first_name" ]
                        student_last_name = current_row[ "last_name" ]

                        # generate student feedback path for student and assignment
                        student_feedback_path = feedback_path + "/" + student_username + "/" + assignment_name

                        print( "----> Student feedback path = " + student_feedback_path )
                        
                        # use try in case student doesn't have feedback
                        try:

                            # list contents of directory
                            file_list = os.listdir( student_feedback_path )

                            # loop over notebook names to convert HTML feedback to PDF.
                            for notebook_name in assignment_notebook_list:

                                # create pandoc file command to convert from HTML to PDF.
                                pandoc_command_list.append( "pandoc" )
                                pandoc_command_list.append( student_feedback_path + "/" + notebook_name + ".html" )
                                pandoc_command_list.append( "-o" )

                                # make and add pdf_file_path.
                                pdf_file_path = student_feedback_path + "/" + notebook_name + ".pdf"
                                pandoc_command_list.append( pdf_file_path )

                                print( "----> Pandoc command: " + str( pandoc_command_list ) )

                                # run the command.
                                pandoc_command_result = subprocess.call( pandoc_command_list )

                                print( "----> command result = " + str( pandoc_command_result ) )

                                # return code 0 = OK
                                if ( pandoc_command_result == 0 ):

                                    # add pdf_file_path to pdf_file_list.
                                    pdf_file_name_list.append( notebook_name + ".pdf" )
                                
                                else:
                                    
                                    # Not OK.  Error.
                                    print( "--------> ERROR code returned - File not added to list." )
                                    
                                #-- END check to see if pandoc executed OK. --#

                            #-- END loop over notbook names. --#

                            '''
                            # == OLD - used to loop over files, slurp up all PDFs.
                            # loop over contents
                            for file_name in file_list:

                                # get lower-cased file name
                                file_name_lower = file_name.lower()

                                # eventually will look for .html files, convert to PDF.
                                # for now, just look for PDF files.

                                # got any PDF files?
                                if ( file_name_lower.rfind( ".pdf" ) == ( len( file_name_lower ) - 4 ) ):

                                    print( "--------> Found PDF file: " + file_name )

                                    # add file path to list.
                                    pdf_file_name_list.append( file_name )

                                #-- END check to see if PDF. --#

                            #-- END loop over files. --#
                            '''

                            # anything in list of PDF files?
                            pdf_file_count = len( pdf_file_name_list )
                            if ( pdf_file_count > 0 ):

                                # there is at least one.  Create a MIME multipart message.
                                multipart_message = MIMEMultipart()

                                # set subject
                                multipart_message[ 'Subject' ] = message_subject

                                # set from
                                multipart_message[ 'From' ] = message_from

                                # set to
                                multipart_message[ 'To' ] = student_email

                                # set Bcc
                                if ( ( message_bcc is not None ) and ( message_bcc != "" ) ):

                                    multipart_message[ 'Bcc' ] = message_bcc
                                    
                                #-- END check to see if message_bcc. --#

                                # preamble - will only appear in non-MIME-aware email reader.
                                multipart_message.preamble = message_subject

                                # add text body of message.
                                complete_message_body = message_body

                                # see if custom message for this student.
                                if ( student_username in custom_message_dict ):

                                    # yes.  Add it.
                                    complete_message_body += "\n" + custom_message_dict[ student_username ] + "\n"

                                #-- END check to see if custom message --#

                                # add signature
                                complete_message_body += message_signature

                                # make text message and append it.
                                text_message = MIMEText( complete_message_body )
                                multipart_message.attach( text_message )

                                # loop over PDF file path list.
                                for file_name in pdf_file_name_list:

                                    # make it into a full on file path:
                                    pdf_file_path = student_feedback_path + "/" + file_name

                                    print( "current file path: " + pdf_file_path )

                                    # get mime type and encoding guesses.
                                    content_type, encoding = mimetypes.guess_type( pdf_file_path )
                                    if content_type is None or encoding is not None:

                                        # No guess could be made, or the file is encoded, so
                                        # use a generic bag-of-bits type.
                                        content_type = 'application/octet-stream'

                                    #-- END check to see if we have a content type, encoding --#

                                    # get main and sub content types.
                                    main_type, sub_type = content_type.split('/', 1)

                                    print( "current content_type: " + content_type )

                                    # open and add PDF file.
                                    with open( pdf_file_path, 'rb' ) as fp:

                                        pdf_message = MIMEApplication( fp.read(), _subtype = sub_type )

                                    #-- END adding PDF to email. --#

                                    # Set the filename's Content-Disposition HTTP header.
                                    pdf_message.add_header( 'Content-Disposition', 'attachment', filename = file_name )
                                    multipart_message.attach( pdf_message )

                                #-- END loop over pdf_file_path_list --#

                                # send message
                                smtp_send_result = smtp_server.send_message( multipart_message )
                                print( "Send result (empty dict is Success!): " + str( smtp_send_result ) )

                            #-- END check to see if PDF file count --#

                        except Exception as e:

                            exception_type, exception_value, exception_traceback = sys.exc_info()
                            print( "Exception caught trying to read \"" + student_feedback_path + "\":" )
                            print( "- args = " + str( e.args ) )
                            print( "- type = " + str( exception_type ) )
                            print( "- value = " + str( exception_value ) )
                            print( "- traceback = " + str( traceback.format_exc() ) )

                        #-- END try-except for file IO for a given student. --#

                    #-- END loop over results --#

                #-- END with smtplib.SMTP( host = smtp_host, port = smtp_port ) as smtp_server: --#
            
            else:
                
                # assignment not OK.
                print( "Problem with assignment.  Can't distribute feedback." )

            #-- END check to see if assignment is OK. --#
                
        except Exception as e:

            exception_type, exception_value, exception_traceback = sys.exc_info()
            print( "Exception caught: " )
            print( "- args = " + str( e.args ) )
            print( "- type = " + str( exception_type ) )
            print( "- value = " + str( exception_value ) )
            print( "- traceback = " + str( traceback.format_exc() ) )

        finally:

            print( "In the 'finally:', cleaning up our mess." )

            # close cursor
            my_cursor.close()

            # close connection
            my_connection.close()

        #-- END try-->except-->finally around database connection. --#

    else:
        
        print( "You must set the assignment name." )
        
    #-- END check to make sure we have an assignment name. --#
    
else:
    
    print( "You must only run this from the nbgrader home directory ( " + nbgrader_home + " )." )

#-- END check to make sure we are in nbgrader home. --#

<hr/>

# Troubleshooting

- Back to the [Table of Contents](#Table-of-Contents)

## `nbgrader formgrade` issues

- Back to the [Table of Contents](#Table-of-Contents)

### CONFIGPROXY_AUTH_TOKEN mismatch between jupyterhub and formgrade

- Back to the [Table of Contents](#Table-of-Contents)

Example output for CONFIGPROXY_AUTH_TOKEN mismatch between server jupyterhub and formgrade:

    $ nbgrader formgrade
    [FormgradeApp | INFO] Proxying /hub/nbgrader/2015-fall-big_data --> http://127.0.0.1:9000
    Traceback (most recent call last):
      File "/usr/local/bin/nbgrader", line 11, in <module>
        sys.exit(main())
      File "/usr/local/lib/python3.4/dist-packages/nbgrader/apps/nbgraderapp.py", line 232, in main
        NbGraderApp.launch_instance()
      File "/usr/local/lib/python3.4/dist-packages/jupyter_core/application.py", line 267, in launch_instance
        return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
      File "/usr/local/lib/python3.4/dist-packages/traitlets/config/application.py", line 592, in launch_instance
        app.start()
      File "/usr/local/lib/python3.4/dist-packages/nbgrader/apps/nbgraderapp.py", line 225, in start
        super(NbGraderApp, self).start()
      File "/usr/local/lib/python3.4/dist-packages/jupyter_core/application.py", line 256, in start
        self.subapp.start()
      File "/usr/local/lib/python3.4/dist-packages/nbgrader/apps/formgradeapp.py", line 125, in start
        parent=self)
      File "/usr/local/lib/python3.4/dist-packages/nbgrader/auth/hubauth.py", line 102, in __init__
        raise Exception('Error while trying to add JupyterHub route. {}: {}'.format(response.status_code, response.text))
    Exception: Error while trying to add JupyterHub route. 403: 

If you see something like this, make sure that you are explicitly setting the CONFIGPROXY_AUTH_TOKEN for the jupyterhub server (preferably in the `c.JupyterHub.proxy_auth_token` variable in `jupyterhub_config.py`), and that you are exporting that same value before starting the formgrade app.

### "attempt to write a readonly database"

- Back to the [Table of Contents](#Table-of-Contents)

If you see an error like this:

    sqlalchemy.exc.OperationalError: (sqlite3.OperationalError) attempt to write a readonly database [SQL: 'INSERT INTO api_tokens (hashed, prefix, user_id) VALUES (?, ?, ?)'] [parameters: ('sha512:16384:09072f984a3396f7:7d01c1f9e2e9bef9abdf71342eac9feaa04757c5845e97dbbcc3ed34f2a49dd92bfc741ad7776cdba87a2fc33410b6bc6be1d0d983cb7c05e9debd45455554a0', 'ab39', 6)]
    
This means your jupyterhub.sqlite database is not able to be written to by the user used to start the formgrade application.  Try adjusting the permissions so that the user who starts formgrade can write to the database.

### "unable to open database file" api_tokens

- Back to the [Table of Contents](#Table-of-Contents)

If you see the following in the output from `nbgrader formgrade`:

    sqlalchemy.exc.OperationalError: (sqlite3.OperationalError) unable to open database file [SQL: 'INSERT INTO api_tokens (hashed, prefix, user_id) VALUES (?, ?, ?)'] [parameters: ('sha512:16384:19f732de1639d068:c517288d4f3716af666657325274c91812ad97e8252f5cf218db2c9f62c4c698a3762f62f27d8fe761f781740d2b525ce58a8a33d91308325761950524dd2aba', 'a151', 6)]
    
This is likely because the folder that contains your `jupyterhub.sqlite` database file is not writeable by the user used to start the formgrade application (sqlite writes temp files in the directory where the database it is using lives).  Try adjusting the permissions on the directory that contains `jupyterhub.sqlite` so that the user who starts formgrade can write to it.

### URLs on formgrade pages refer to proxy IP address, not public IP or domain name

- Back to the [Table of Contents](#Table-of-Contents)

If you get past the above issues and can authenticate, but when the page loads, it is unstyled and all the links point to `localhost:8000` or `127.0.0.1:8000` rather than the actual public address of your server (including javascript and CSS, which won't load, so the page will look unstyled and won't work right), then you need to make sure that the `c.HubAuth.hub_address` property is set to the public IP address or domain name for your server in `nbgrader_config.py` for the instructor user.  Example:

    c.HubAuth.hub_address = 'bigdataforsocialscience.com'

If you have SSL configured, you'll need to set a different variable, you'll need to instead set the `c.HubAuth.hub_base_url` to the entire URL of the jupyterhub server, including protocol and port number.  This value takes priority over the `c.HubAuth.hub_address`, so no need to set that when this is set.  Example:

    #c.HubAuth.hub_base_url = 'https://localhost:8000'
    c.HubAuth.hub_base_url = 'https://bigdataforsocialscience.com:8000'

### Notebook file not found when you try to grade a given submission

- Back to the [Table of Contents](#Table-of-Contents)

If when you try to grade a given submission you get a message that a given submission's notebook file could not be found, there is a chance that student in question (which is obfuscated so the grading is blind) might have re-named the notebook.

If this is the case, to resolve, you need to:

- look in `<nbgrader_home>/submitted`, in each student's folder, in their folder for the assignment in question, to see if you can find the misnamed notebook.
- if you find it, fix the name.
- in `<nbgrader_home>/autograded`, to get this change reflected in the formgrade application, you'll then need to remove the folder for that assignment from the student's folder, then re-autograde.

## notebook issues

- Back to the [Table of Contents](#Table-of-Contents)

### nbgrader solution can not be in a normal cell

- Back to the [Table of Contents](#Table-of-Contents)

If, when you run `nbgrader assign`, you get the error:

    RuntimeError: Solution region detected in a non-solution cell; please make sure all solution regions are within solution cells
        
It could be because you created your notebook with cells that contain nbgrader (for example, `### BEGIN SOLUTION ###` and `### END SOLUTION ###`), but aren't specified as nbgrader cells using the "Create Assignment" cell toolbar.  Make sure that all cells that have solutions are specified correctly as Assignment cells using the toolbar.

### grade cell point value must be greater than 0

- Back to the [Table of Contents](#Table-of-Contents)

If, when you run `nbgrader assign`, you get the error:

    RuntimeError: Point value for grade cell correct_sum_of_squares is invalid:
        
Make sure that each of your graded Assignment cells has a point value that is greater than 0.  It can be a decimal like 0.5, but 0 is invalid.

### make sure you run code cells that just contain a function definition before calling the function

- Back to the [Table of Contents](#Table-of-Contents)

If you don't, and then you try to run the function, you'll get a really confusing and amazing explosion of exception messages and stack trace.  Be advised, on the watch for students that do this.

## gradebook database issues

- Back to the [Table of Contents](#Table-of-Contents)

### corruption of tables that hold auto-grading

- Back to the [Table of Contents](#Table-of-Contents)

If your database is corrupted, including information on auto-graded assignments, here is code you can run to re-create the database structure needed to auto-grade.  It doesn't handle orhpaned records and you'll lose any grading recorded in the lost data, but it should at least make it so you can auto-grade over again.

In [ ]:
from nbgrader.api import Gradebook, MissingEntry, Grade, SubmittedNotebook, Comment
import sqlite3

# declare variables
student_id_list = []
current_student_id = ""

# make a connection - pass the name of the database file.
my_connection = sqlite3.connect( 'gradebook.db' )

# set row_factory that returns values mapped to column names
#   as well as in an ordered list
my_connection.row_factory = sqlite3.Row

# then, make a cursor.
my_cursor = my_connection.cursor()

sql_string = "SELECT * FROM student ORDER BY last_name ASC, first_name ASC;"
#sql_string = "SELECT * FROM student WHERE id IN ( 'jmorgan' ) ORDER BY last_name ASC, first_name ASC;"
#sql_string = "SELECT * FROM student WHERE id NOT IN ( 'cjones', 'aemad' ) ORDER BY last_name ASC, first_name ASC;"

# Actually use cursor here...
result_set = my_cursor.execute( sql_string )

# output results
print( "Student list:" )

# loop over students in result set.
student_counter = 0
for current_row in result_set:

    # increment student counter
    student_counter += 1

    # initialize variables
    current_student_id = ""
    student_email = ""
    student_first_name = ""
    student_last_name = ""

    print( "\n\n==> Student " + str( student_counter ) + " = " + current_row[ "first_name" ] + " " + current_row[ "last_name" ] + " ( " + current_row[ "id" ] + " ) - " + current_row[ "email" ] )

    # get student information
    current_student_id = current_row[ "id" ]
    student_email = current_row[ "email" ]
    student_first_name = current_row[ "first_name" ]
    student_last_name = current_row[ "last_name" ]
    
    # add ID to student_id_list
    student_id_list.append( current_student_id )

#-- END loop over students --#
    
# close cursor and connection
my_cursor.close()
my_connection.close()

# now, clean up assignments for each student.

gb = Gradebook( "sqlite:///gradebook.db" )

#assignment_name = '02. Database Basics'
assignment_name = ''
student_id = ''

if ( ( assignment_name is not None ) and ( assignment_name != "" ) ):

    for student_id in student_id_list:

        # get submission
        submission = gb.update_or_create_submission(assignment_name, student_id)

        for notebook in submission.assignment.notebooks:
            try:
                # query the database for the submitted notebook...
                nb = gb.find_submission_notebook(notebook.name, assignment_name, student_id)
            except:
                # if the submitted notebook doesn't exist, create it
                nb = SubmittedNotebook(notebook=notebook, assignment=submission)

            for grade_cell in notebook.grade_cells:
                try:
                    # query the database for the grade...
                    gb.find_grade(grade_cell.name, notebook.name, assignment_name, student_id)
                except MissingEntry:
                    # if the grade doesn't exist, create it
                    Grade(cell=grade_cell, notebook=nb)

            for solution_cell in notebook.solution_cells:
                try:
                    # query the database for the comment...
                    gb.find_comment(solution_cell.name, notebook.name, assignment_name, student_id)
                except MissingEntry:
                    # if the comment doesn't exist, create it
                    Comment(cell=solution_cell, notebook=nb)

        gb.db.add(submission)
        gb.db.commit()

    #-- END loop over students --#

else:
    
    print( "No assignment, no work done." )

#-- END check to see if assignment. --#